In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

import datetime

In [3]:
import warnings

warnings.filterwarnings('ignore')

### Load data

In [4]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


###  Split train, valid, test

In [5]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


### HyperParameter

In [34]:
EPOCHS = 50
STEPS_PER_EPOCH = 5
BATCHSIZE = 32

### Create Dataset

In [35]:
def input_fn(df, label, shuffle=True, batch_size=32):
    df = df.copy()
    target = df.pop(label)
    ds = tf.data.Dataset.from_tensor_slices((dict(df), target))
    ds = ds.cache()
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds.repeat() 

In [38]:
train_spec = tf.estimator.TrainSpec(input_fn=lambda: input_fn(train, 'target', shuffle=True, batch_size=BATCHSIZE),
                                    max_steps=STEPS_PER_EPOCH * EPOCHS)
eval_spec = tf.estimator.EvalSpec(input_fn=lambda: input_fn(val, 'target', shuffle=False, batch_size=BATCHSIZE), name='valid1',
                                  steps=1)

### Create Feature Columns

In [8]:
feature_columns = []

# 数值列
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
    feature_columns.append(feature_column.numeric_column(header))

# 分桶列
age = feature_column.numeric_column("age")
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# 分类列
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# 嵌入列
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# 组合列
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

In [9]:
example_batch = next(iter(tr_ds))[0]

In [10]:
# 用于创建一个特征列
# 并转换一批次数据的一个实用程序方法
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy().shape)

In [11]:
demo(feature_columns)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

(32, 1029)


### Model

#### cross layer

In [12]:
import tensorflow.keras.backend as K

In [13]:
class CrossLayer(tf.keras.layers.Layer):
    def __init__(self, input_dim, output_dim=30, **kwargs):
        self.input_dim = input_dim
        self.output_dim = output_dim
        super(CrossLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(self.input_dim, self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)
        super(CrossLayer, self).build(input_shape)

    def call(self, x):
        a = K.pow(K.dot(x,self.kernel), 2)
        b = K.dot(K.pow(x, 2), K.pow(self.kernel, 2))
        return K.mean(a-b, 1, keepdims=True)*0.5

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

#### feature column 进 keras

In [14]:
class deepFM(tf.keras.Model):
    def __init__(self):
        super(deepFM, self).__init__()
        self.feat_layer = tf.keras.layers.DenseFeatures(feature_columns)
        self.deep = tf.keras.layers.Dense(128, activation='relu')
        self.cs = CrossLayer(1029)   ### 1029 is dimension of feature
        self.concat = tf.keras.layers.Concatenate(axis=-1)
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')    

    def call(self, x):
        x = self.feat_layer(x)
        deep = self.deep(x)
        cross = self.cs(x)
        x = self.concat([deep, cross])
        x = self.dense(x)
        return x

In [30]:
def my_model_fn(features, labels, mode):
    model = deepFM()

    training = (mode == tf.estimator.ModeKeys.TRAIN)
    loss_obj = tf.keras.losses.BinaryCrossentropy()
    predictions = model(features, training=training)

    # Get both the unconditional losses (the None part)
    # and the input-conditional losses (the features part).
    reg_losses = model.get_losses_for(None) + model.get_losses_for(features)
    total_loss=loss_obj(labels, predictions) #+ tf.math.add_n(reg_losses)

    # Upgrade to tf.keras.metrics.
    accuracy_obj = tf.keras.metrics.Accuracy(name='acc_obj')
    accuracy = accuracy_obj.update_state(
      y_true=labels, y_pred=tf.math.argmax(predictions, axis=1))

    train_op = None
    if training:
        # Upgrade to tf.keras.optimizers.
        optimizer = tf.keras.optimizers.Adam()
        # Manually assign tf.compat.v1.global_step variable to optimizer.iterations
        # to make tf.compat.v1.train.global_step increased correctly.
        # This assignment is a must for any `tf.train.SessionRunHook` specified in
        # estimator, as SessionRunHooks rely on global step.
        optimizer.iterations = tf.compat.v1.train.get_or_create_global_step()
        # Get both the unconditional updates (the None part)
        # and the input-conditional updates (the features part).
        update_ops = model.get_updates_for(None) + model.get_updates_for(features)
        # Compute the minimize_op.
        minimize_op = optimizer.get_updates(
            total_loss,
            model.trainable_variables)[0]
        train_op = tf.group(minimize_op, *update_ops)

    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=total_loss,
        train_op=train_op,
        eval_metric_ops={'Accuracy': accuracy_obj})

In [41]:
run_config = tf.estimator.RunConfig(log_step_count_steps=50)

In [42]:
# Create the Estimator & Train.
estimator = tf.estimator.Estimator(model_fn=my_model_fn, config=run_config)
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpo8gfk0cg', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 50, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and

({'Accuracy': 0.90625, 'loss': 0.26454523, 'global_step': 250}, [])